In [1]:
import json
import numpy as np
import math
import os
import cv2
import open3d as o3d
import natsort 
import shutil
from pyquaternion import Quaternion
import sys
import random
import copy

output_path='/mnt/nas3/Data/PCD/indoor_parking/fov_test/custom/'
asset_path ='/mnt/nas3/Data/PCD/indoor_parking/asset/'
label_path ='/mnt/nas3/Data/PCD/indoor_parking/labels/'


train_ratio = 0.8
label_margin = 10
class_names=[]
max_int = sys.maxsize
parameter_list = []
total_label=0
filtered_label=0
multi_modal = False
data_dir_num_thres = 2

#Q : FOV아니고 whole scene label?
# TODO : float32저장, np.load, no label 삭제, label이 띄어쓰기는 _등으로 연결

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
asset_path_list = os.listdir(asset_path)
for path in asset_path_list:
    files = os.listdir(asset_path+path)
    for file in files: 
        if '.jpg' in file or '.png' in file:
            multi_modal = True
            break
    if multi_modal is False:
        break


In [3]:
'''
convert calibration info to kitti form
'''
def convert_calib(calib_path):
#     train_output_path = output_path+'calib/'

#     try:
#         if not os.path.exists(train_output_path):
#             os.makedirs(train_output_path)
#     except OSError:
#         print("Error: Failed to create the directory.")

    try:
        file = open(calib_path)
    except:
        print("Error: Failed to open the file")
   
    jsonString = json.load(file)
    jsonArray = jsonString.get("manifest").get('frames')
    idx=0
    for list in jsonArray:
        imageArray = list.get("images")
        if len(imageArray) == 0:
            multi_modal = False
            return None,None
        for camArray in imageArray:
            positionArray = camArray.get('position')
            pose_x = positionArray.get('x')
            pose_y = positionArray.get('y')
            pose_z = positionArray.get('z')
            
            quaternionArray = camArray.get('heading')
            qw = quaternionArray.get('qw')
            qx = quaternionArray.get('qx')
            qy = quaternionArray.get('qy')
            qz = quaternionArray.get('qz')
            rot_matrix = Quaternion(qw,qx,qy,qz).rotation_matrix
            rot_matrix = np.linalg.inv(rot_matrix)
            translation_matrix = np.array([pose_x,pose_y,pose_z]).reshape(3,1)
            kitti_style_extrinsic = np.concatenate([rot_matrix, -np.matmul(rot_matrix, translation_matrix).reshape(3, 1)], axis=1)
            cx = camArray.get('cx')
            cy = camArray.get('cy')
            skew = camArray.get('skew')
            fx = camArray.get('fx')
            fy = camArray.get('fy')
            
            pro_cx=camArray.get('cx')
            pro_fx=camArray.get('fx')
            pro_skew=camArray.get('skew')
            pro_cy=camArray.get('cy')
            pro_fy=camArray.get('fy')

            projectionArray = camArray.get('new_camera_matrix')
            pro_cx=projectionArray.get('cx')
            pro_fx=projectionArray.get('fx')
            pro_skew=projectionArray.get('skew')
            pro_cy=projectionArray.get('cy')
            pro_fy=projectionArray.get('fy')
            

            intrinsic = np.array([fx,skew,cx,0.0,fy,cy,0.0,0.0,1.0]).reshape(3,3)
            extrinsic = np.array([kitti_style_extrinsic[0,0], kitti_style_extrinsic[0,1], kitti_style_extrinsic[0,2], kitti_style_extrinsic[0,3], kitti_style_extrinsic[1,0], kitti_style_extrinsic[1,1], kitti_style_extrinsic[1,2], kitti_style_extrinsic[1,3], kitti_style_extrinsic[2,0], kitti_style_extrinsic[2,1], kitti_style_extrinsic[2,2], kitti_style_extrinsic[2,3]]).reshape(3,4)

    return intrinsic, extrinsic





In [4]:

def filter_cameraFOV_points(points,height,width,intrinsic,extrinsic):
    points = points[:,:3]
    points_shape = points.shape
    ones = np.ones((points_shape[0], 1))
    points_homo = np.concatenate([points, ones], axis=1)
    points_cam = np.dot(extrinsic, points_homo.T).T
    points_prj = np.dot(intrinsic, points_cam.T).T
    points_prj /= points_prj[:, 2:3]
    eff_inds = (
        (points_prj[:, 0] < width+label_margin)
        & (points_prj[:, 0] >= 0)
        & (points_prj[:, 1] < height+label_margin)
        & (points_prj[:, 1] >= 0)
        & (points_cam[:, 2] >=0)
    )
    eff_inds_where = np.where(eff_inds)[0]
#     print(eff_inds_where)
    points[eff_inds_where,:]
    return points

In [5]:
def convert_pcd_to_npy(file_path,save_path,height=None,width=None,intrinsic=None,extrinsic=None):
    try:
        points = o3d.read_point_cloud(file_path)
        points = np.asarray(point_cloud.points)[:, :3]
        if multi_modal:
            points = filter_cameraFOV_points(points,height,width,intrinsic,extrinsic)

        zeros = np.zeros(np.shape(points)[0])
        zeros = np.reshape(zeros, (-1, 1))
        points = np.concatenate([points, zeros], axis=1)
        np.save(save_path, points)
    except:
        print("can not convert bin to npy")
        print(file_path)
        
def convert_bin_to_npy(file_path,save_path,height=None,width=None,intrinsic=None,extrinsic=None):
    try:
        points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        if multi_modal:
            points = filter_cameraFOV_points(points,height,width,intrinsic,extrinsic)
            zeros = np.zeros(np.shape(points)[0])
            zeros = np.reshape(zeros, (-1, 1))
            points = np.concatenate([points, zeros], axis=1)
        np.save(save_path, points)
    except:
        print("can not convert bin to npy")
        print(file_path)
    

In [6]:
'''
move pcd file to each folder and rename as order
'''
def copy_pcd(mode,path,output_path,idx,height=None,width=None,intrinsic=None,extrinsic=None):       
    try:
        if not os.path.exists(output_path+'points/'):
            os.makedirs(output_path+'points/')
    except OSError:
        print("Error: Failed to create the directory.")
   
    if mode == "pcd":
        output_points_path = output_path+'points/'
        convert_pcd_to_npy(path, output_points_path+str(idx).zfill(6)+".npy",height,width,intrinsic,extrinsic)  
    elif mode == "bin":
        output_points_path = output_path+'points/'    
        convert_bin_to_npy(path, output_points_path+str(idx).zfill(6)+".npy",height,width,intrinsic,extrinsic)          



In [7]:

def filter_cameraFOV_labels(x,y,z,calib_idx):

    intrinsic = parameter_list[calib_idx]['intrinsic']
    extrinsic = parameter_list[calib_idx]['extrinsic']
    width = parameter_list[calib_idx]['width']
    height = parameter_list[calib_idx]['height']
        

    label_pose = np.array([x,y,z,1])
    label_pose_cam = np.dot(extrinsic,label_pose.T).T
    label_pose_prj = np.dot(intrinsic,label_pose_cam.T).T
    label_pose_prj /= label_pose_prj[2:3]
    eff_inds = (
        (label_pose_prj[0] < width+label_margin)
        & (label_pose_prj[0] >= 0)
        & (label_pose_prj[1] < height+label_margin)
        & (label_pose_prj[1] >= 0)
        & (label_pose_cam[2] >=0)
    )
    return eff_inds
    

In [8]:
'''
convert label info to custom form

format: [x y z dx dy dz heading_angle category_name]

'''
def preprocess_label(label_output,frame_num):
    label_output = label_output+"labels/"
    try:
        if not os.path.exists(label_output):
            os.makedirs(label_output)
    except OSError:
        print("Error: Failed to create the directory.")
    for i in range(frame_num):
        try:
            f = open(label_output+str(label_idx+i).zfill(6)+".txt", 'w')
            f.close()
        except:
            print("Fail to create file")

def convert_label_suite2custom(label_path,label_output, label_idx): 
    label_output = label_output+"labels/"
    print(label_path)
    try:
        file = open(label_path)
    except:
        print("Error: Failed to open the file")


    jsonString = json.load(file)   
    # frame_num = jsonString.get("label").get('asset').get("info").get("frame_count")


    try:
        jsonArray = jsonString.get('objects')
    except:
        print("can't find objects")
        # return label_idx
        return
    
    # calib_path = output_path+'training/calib/'
    for iter_num, list in enumerate(jsonArray):
        frames = list.get("frames")
        class_name = list.get('class_name')
        if class_name == "Wall" or class_name == 'Undefined Object' or class_name =='Undefined Stuff' or class_name=="No Parking Sign":
            continue
        if class_name == 'Traffic Cone':
            class_name = 'Traffic_Cone'
        if class_name =="Shopping Cart":
            class_name = 'Shopping_Cart'
        if class_name =="Tow-Wheeled Vehicle":
            class_name = 'Tow_Wheeled_Vehicle'
        if class_name =="Two-Wheeled Vehicle":
            class_name = 'Two_Wheeled_Vehicle'
        global class_names
        if (class_name in class_names) is not True:
            
            class_names.append(class_name)
            
        for frame in frames:
            global filtered_label
            filtered_label+=1
            idx = frame.get('num')
            
            ann = frame.get('annotation').get('coord')
            height = ann.get('size').get('z')
            width = ann.get('size').get('y')
            length = ann.get('size').get('x')
            # TODO : check coordinate , lidar or cam? -> cam
            
            x = ann.get('position').get('x')
            y = ann.get('position').get('y')
            z = ann.get('position').get('z')
            if not filter_cameraFOV_labels(x,y,z,label_idx+idx):
                print(x,y,z)
                print("label is not in FOV")
                continue
            yaw_pitch_roll = Quaternion(
                ann.get('rotation_quaternion').get('w'),
                ann.get('rotation_quaternion').get('x'),
                ann.get('rotation_quaternion').get('y'),
                ann.get('rotation_quaternion').get('z'),
            ).yaw_pitch_roll
            yaw = yaw_pitch_roll[0]
#             yaw = np.round_(yaw_pitch_roll[0],5)
            global total_label
            total_label+=1
            f = open(label_output+str(label_idx+idx).zfill(6)+".txt", 'a')
            f.write(f'{x} {y} {z} {length} {width} {height} {yaw} {class_name}\n')
            f.close()
    return

                           

In [9]:
def test(num):
    return num+1
    

In [10]:
###calib, file movement exec###

asset_path_list = os.listdir(asset_path)
asset_path_list = natsort.natsorted(asset_path_list)


# print(asset_path_list)
pcd_idx=0

num_frame=[]
num_frame.append(0)
thres_flag=False
for data_cnt,path in enumerate(asset_path_list):
    
    files = os.listdir(asset_path+path)
    files = natsort.natsorted(files)
    json_file_path=None
    for file in files:        
        if multi_modal:
            json_file_path = os.path.join(asset_path+path,'manifest.json')
            intrinsic, extrinsic=convert_calib(json_file_path) 
            
        if '.bin' in file:
            pcd_file_path = os.path.join(asset_path+path, file)
            if multi_modal:
                image_path =os.path.join(asset_path+path, files[-2])
                image = cv2.imread(image_path)
                height, width = image.shape[:2]
                parameter_list.append({'intrinsic':intrinsic,'extrinsic':extrinsic,'width':width,'height':height})
            copy_pcd('bin',pcd_file_path,output_path,pcd_idx,height,width,intrinsic,extrinsic)
            pcd_idx+=1
            
            
        if '.pcd' in file:
            pcd_file_path = os.path.join(asset_path+path, file)
            if multi_modal:
                image_path =os.path.join(asset_path+path, files[-2])
                image = cv2.imread(image_path)
                image = cv2.imread(image_path)
                height, width = image.shape[:2]
                parameter_list.append({'intrinsic':intrinsic,'extrinsic':extrinsic,'width':width,'height':height})
            copy_pcd('pcd',pcd_file_path,output_path,pcd_idx,height,width,intrinsic,extrinsic)
            pcd_idx+=1
               
    if data_dir_num_thres <= data_cnt:
        break
    num_frame.append(pcd_idx)


   
          

In [11]:
print(len(parameter_list))

300


In [12]:
# num_frame
pcd_idx

300

In [13]:
# #####label exec
# import time
# label_list = os.listdir(label_path)
# label_list = natsort.natsorted(label_list)

# label_idx=0

# preprocess_label(output_path,pcd_idx)


# start = time.time()

# idx=0
# for label_data_cnt,label in enumerate(label_list):
#     print(idx)
#     if asset_path_list[idx] == label.split('.')[0]:

#         json_file_path = os.path.join(label_path, label)
# #         file = open(json_file_path)
# #         jsonString = json.load(file)
#         convert_label_suite2custom(json_file_path,output_path, num_frame[idx])
#         if data_dir_num_thres <= idx:
#             break
#         idx+=1
# delta_t = time.time() - start
# print("Total Time :",delta_t)



In [14]:
num_frame
# 1.151613712310791

[0, 100, 200]

In [16]:
#####label exec

label_list = os.listdir(label_path)
label_list = natsort.natsorted(label_list)

label_idx=0

preprocess_label(output_path,pcd_idx)



idx=0
for label_data_cnt,label in enumerate(label_list):
    print(idx)
    if asset_path_list[idx] == label.split('.')[0]:

        json_file_path = os.path.join(label_path, label)
        file = open(json_file_path)
        jsonString = json.load(file)
        convert_label_suite2custom(json_file_path,output_path, num_frame[idx])
        if data_dir_num_thres <= idx:
            break
        idx+=1
    


0
/mnt/nas3/Data/PCD/indoor_parking/labels/0a5f9ed1-0268-4bff-9119-96ef08ca634f.json
-2.4458891197864885 1.9820872455208909 -1.0959908016216375
label is not in FOV
3.626006969440372 3.275878749462728 -1.1278728607189812
label is not in FOV
-3.362647556445652 2.8201711200422284 -1.4644556181173987
label is not in FOV
-3.3235951808234865 2.9186733701551315 -1.2999591971752409
label is not in FOV
-10.045769740462356 9.649457560470768 -1.221273372153377
label is not in FOV
-10.16923724607816 9.272515083373614 -1.2341618076876402
label is not in FOV
-10.260528452087266 9.003886876995292 -1.2470502390873688
label is not in FOV
-10.312039052321477 8.63155162018137 -1.0937469889950973
label is not in FOV
-10.321818849020703 8.45963809055248 -1.1001913894564612
label is not in FOV
-10.380425702444304 8.016041958472108 -1.211099132144626
label is not in FOV
-10.341795639584644 7.721931270056405 -1.2175435174571305
label is not in FOV
-10.383183494080285 7.319681725266109 -1.2073684410064511
labe

-5.211971039911482 4.450898854975886 -1.3919827274833882
label is not in FOV
-4.973460826559526 4.508634264204451 -1.406979287119983
label is not in FOV
-2.959778912162324 2.3665442522040716 -1.3951197862625122
label is not in FOV
-2.5606812513985306 1.9503152518401845 -1.4045676011571515
label is not in FOV
-2.5598015034599917 1.549969591190209 -1.362304471459795
label is not in FOV
3.615492444868659 3.0208271997712517 -1.3989300827577482
label is not in FOV
3.775572252253941 3.297264608038705 -1.4207263926910287
label is not in FOV
2.6406022726428984 2.0116446350800317 -1.4495967300223391
label is not in FOV
-11.334645563013414 10.08712776160679 -0.3875427842140198
label is not in FOV
2
/mnt/nas3/Data/PCD/indoor_parking/labels/0a7cdfd5-6387-4f3a-833b-b928622ed8b8.json
-5.172592324154977 4.748427236055488 0.0414077940696449
label is not in FOV
-4.797479721563928 4.502640781438784 -0.9772257039056099
label is not in FOV
-4.70761916396446 3.4861148183898387 -1.0424431086431285
label is 

In [ ]:
def test(a):
    return a
# https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments

In [ ]:
from multiprocessing import Pool
p = Pool(4)
a=[1,2,3]
b=[1,2,3]
c=[1,2,3]
p.map(test,a)

In [17]:
num_frame_list

[0, 100, 200]

In [18]:
print(filtered_label)
print(total_label)

2825
2601


In [19]:
def delete_empty_asset(asset_path,empty_label_list):
#     label_path_list = os.listdir(label_path)
    empty_label_list = natsort.natsorted(empty_label_list)
    asset_path_list = os.listdir(asset_path)
    asset_path_list = natsort.natsorted(asset_path_list)
    empty_cnt=0
    print(empty_label_list)
    for asset in asset_path_list:
#         if empty_cnt == len(empty_label_list):
#             break
        if empty_cnt<len(empty_label_list) and asset.split('.')[0] == empty_label_list[empty_cnt] :
            os.remove(asset_path+asset)
            empty_cnt+=1
            continue
        
        os.rename(asset_path+asset, asset_path+str(int(asset.split('.')[0])-empty_cnt).zfill(6)+'.npy')
    print(empty_cnt)
  
    
    
   

In [20]:
def check_empty_label(label_path):
    label_path_list = os.listdir(label_path)
    label_path_list = natsort.natsorted(label_path_list)
    empty_num=0
    empty_list =[]
    for label in label_path_list:
        with open(label_path+label, 'r') as f:
            lines = f.readlines()
            if len(lines)==0:
                empty_num+=1
                empty_list.append(label.split('.')[0])
                os.remove(label_path+label)
                continue
        os.rename(label_path+label, label_path+str(int(label.split('.')[0])-empty_num).zfill(6)+'.txt')
    return empty_list

In [21]:
print(len(os.listdir(output_path+'labels/')))
print(len(os.listdir(output_path+'points/')))

300
300


In [22]:
empty_label_list = check_empty_label(output_path+'labels/')
delete_empty_asset(output_path+'points/',empty_label_list)
print(len(empty_label_list))

['000287', '000288', '000289']
3
3


In [23]:
print(len(os.listdir(output_path+'labels/')))
print(len(os.listdir(output_path+'points/')))

297
297


In [24]:
label_list = os.listdir(output_path+'labels/')
label_path_list = natsort.natsorted(label_list)
for idx,label in enumerate(label_path_list):
    if idx != int(label.split('.')[0]):
        print(idx)
        

In [25]:
asset_list = os.listdir(output_path+'points/')
asset_list = natsort.natsorted(asset_list)
for idx,asset in enumerate(asset_list):
    if idx != int(asset.split('.')[0]):
        print(idx)
        

In [26]:
#### split train, test
total_num=len(os.listdir(output_path+'points/'))

### shffle
total_num_list = list()
for i in range(total_num):
    total_num_list.append(i)
random.shuffle(total_num_list)

### mkdir
try:
    if not os.path.exists(output_path+'ImageSets/'):
        os.makedirs(output_path+'ImageSets/')
        print("generation")
except OSError:
    print("Error: Failed to create the directory.")
    
f2 = open(output_path+'ImageSets/'+'train'+".txt", 'w')
f3 = open(output_path+'ImageSets/'+'val'+".txt", 'w')
for idx,num in enumerate(total_num_list):
    if idx < total_num*train_ratio:
        f2.write(f'{str(num).zfill(6)}\n')
    else:
        f3.write(f'{str(num).zfill(6)}\n')
    idx+=1
f2.close()
f3.close()


generation


In [27]:
class_names

['Human',
 'Wheelchair',
 'Vehicle',
 'Stroller',
 'Pillar',
 'Two_Wheeled_Vehicle',
 'Traffic_Cone',
 'Shopping_Cart']